In [158]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, roc_curve, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt



In [145]:
static_variables = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
static_variables_we_want = ['Age', 'Gender', 'Height', 'Weight']
all_variables = ['Weight', 'Age', 'TroponinI', 'DiasABP', 'MechVent', 'HCO3', 'Cholesterol', 'HCT', 'SaO2', 'WBC', 'SysABP', 'Urine', 'ICUType', 'Gender', 'ALP', 'Creatinine', 'K', 'AST', 'Glucose', 'RespRate', 'MAP', 'FiO2', 'BUN', 'Na', 'Bilirubin', 'TroponinT', 'PaCO2', 'Height', 'GCS', 'HR', 'pH', 'PaO2', 'Lactate', 'ALT', 'NISysABP', 'RecordID', 'Platelets', 'Temp', 'Mg', 'NIDiasABP', 'Albumin', 'NIMAP']
dyn_variables = [x for x in all_variables if x not in static_variables]
dyn_variables.append('Weight_VAR')

print(len(dyn_variables), len(static_variables_we_want))

37 4


In [146]:
train_df = pd.read_parquet("data/set-a_no_nan.parquet")
val_df = pd.read_parquet("data/set-b_no_nan.parquet")
test_df = pd.read_parquet("data/set-c_no_nan.parquet")

outcomes_a_df = pd.read_csv("data/Outcomes-a.txt", sep=",")
outcomes_b_df = pd.read_csv("data/Outcomes-b.txt", sep=",")
outcomes_c_df = pd.read_csv("data/Outcomes-c.txt", sep=",")

In [147]:
tmp_outcomes = [outcomes_a_df, outcomes_b_df, outcomes_c_df]

data_df = [train_df, val_df, test_df]

outcome_labels = []
for i in range(3):
    outcome_records = tmp_outcomes[i]['RecordID'].unique().astype(int)
    train_records = data_df[i]['RecordID'].unique().astype(int)
    assert (outcome_records - train_records == 0).all(), "Mismatch: expected difference of 0 between outcome_records and train_records."

    outcome_labels.append(tmp_outcomes[i]["In-hospital_death"].values) # 1 if patient died in hospital, 0 otherwise
    print(outcome_labels[i].shape, type(outcome_labels[i]))
print("All records in outcomes and data are in same order")

(4000,) <class 'numpy.ndarray'>
(4000,) <class 'numpy.ndarray'>
(4000,) <class 'numpy.ndarray'>
All records in outcomes and data are in same order


In [148]:
# Convert dfs into numpy arrays
def convert_df_to_np(df):
    dfs = []
    for record_id in df['RecordID'].unique():
        df_tmp = df[df['RecordID'] == record_id]
        df_tmp = df_tmp.drop(columns=['RecordID', "Time"])
        arr = df_tmp.to_numpy()
        dfs.append(arr)

    # convert list of dfs to list of tensors
    train_data = np.array(dfs)
    return train_data

train_data = convert_df_to_np(data_df[0])
val_data = convert_df_to_np(data_df[1])
test_data = convert_df_to_np(data_df[2])

# Standardize data
mean = train_data.mean(axis=(0,1), keepdims=True)
std = train_data.std(axis=(0,1), keepdims=True)
std[std == 0] = 1.0


# 2. Standardize data
train_data = (train_data - mean) / std
val_data = (val_data - mean) / std  
test_data = (test_data - mean) / std  #

print(train_data.shape, val_data.shape, test_data.shape)

(4000, 49, 41) (4000, 49, 41) (4000, 49, 41)


In [149]:

class MedicalTimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [150]:
train_dataset = MedicalTimeSeriesDataset(train_data, outcome_labels[0])
val_dataset = MedicalTimeSeriesDataset(val_data, outcome_labels[1])
test_dataset = MedicalTimeSeriesDataset(test_data, outcome_labels[2])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [151]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_length=49):
        super().__init__()

        pe = torch.zeros(max_length, d_model) # max_length: number tokens, d_model: dimension of each token (embedding dim)

        position = torch.arange(0, max_length).unsqueeze(1) # shape (max_length, 1)

        div_term = torch.exp(
            torch.arange(0, d_model, 2)* -(torch.log(torch.tensor(10000.0)) / d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))
        self.pe: torch.Tensor

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

class TimeSeriesTransformer(nn.Module):

    def __init__(self, feature_dim = 41, d_model = 128, nhead = 4, num_layers = 2,dropout=0.1) -> None:
        super().__init__()

        self.input_proj = nn.Linear(feature_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=False)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.d_model = d_model # Store d_model for classifier input dim

        self.classifier = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
            )

    def forward(self, x):
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2) # transformer expects (seq_len, batch, features)
        x = self.transformer_encoder(x)

        # --- Global Average Pooling ---
        # Average across the sequence length dimension (dim=0)
        x = x.mean(dim=0) # (batch, d_model) e.g., (64, 128)
        # --- Alternative: Use Last Time Step ---
        # x = x[-1, :, :] # (batch, d_model) e.g., (64, 128)
        # ---------------------------------------
        x = self.classifier(x)
        return x

In [166]:
model = TimeSeriesTransformer(num_layers=3)

device = torch.device("cuda" if torch.cuda.is_available() else  "mps" if torch.mps.is_available() else "cpu")
model.to(device)


n_samples = len(outcome_labels[0])
n_positives = sum(outcome_labels[0])
n_negatives = n_samples - n_positives

if n_positives > 0:
    pos_weight_val = n_negatives / n_positives
else:
    pos_weight_val = 1.0 # Or handle as an error if appropriate
pos_weight_val = 4.0

print(f"Calculated pos_weight: {pos_weight_val:.4f} (Negatives={n_negatives}, Positives={n_positives})")
pos_weight = torch.tensor([pos_weight_val], dtype=torch.float32).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01) # Added weight decay


# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Validation/testing function
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = [] # Store raw outputs (logits)
    all_targets = [] # Store targets

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch).squeeze() # raw logits

            # Ensure y_batch has the same shape as outputs if squeeze() removed a dim
            if outputs.dim() == 0: # Handle batch size of 1 if squeezed to scalar
                 outputs = outputs.unsqueeze(0)
            if y_batch.dim() > outputs.dim(): # Ensure y_batch has same shape as output
                y_batch = y_batch.squeeze()
            elif outputs.dim() > y_batch.dim(): # This shouldn't happen with .squeeze() above but safety check
                outputs = outputs.squeeze() # Re-try squeeze if needed

            # Ensure shapes match before loss calculation
            if outputs.shape != y_batch.shape:
                 # This indicates a potential issue elsewhere, maybe with batch size 1 handling
                 print(f"Shape mismatch: outputs {outputs.shape}, y_batch {y_batch.shape}")
                 # Decide how to handle: skip batch, reshape if possible, etc.
                 # For now, we'll just report loss can't be computed for this batch
                 continue # Skip this batch if shapes don't match


            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            all_outputs.extend(outputs.cpu().numpy())
            all_targets.extend(y_batch.cpu().numpy())

    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)

    # Calculate metrics
    avg_loss = total_loss / len(loader)

  # Apply sigmoid to logits to get probabilities for thresholding
    probs = 1 / (1 + np.exp(-all_outputs)) # Sigmoid function
    preds_labels = (probs >= 0.5).astype(int) # Threshold probabilities

    # Ensure targets are integers for accuracy score
    all_targets = all_targets.astype(int)

    accuracy = accuracy_score(all_targets, preds_labels)
    # ROC AUC can be calculated directly from logits (or probabilities)
    # Check if there's more than one class present in targets for ROC AUC
    if len(np.unique(all_targets)) > 1:
        roc_auc = roc_auc_score(all_targets, all_outputs) # Use logits directly
    else:
        roc_auc = 0.5 # Or np.nan, indicating AUC is not defined
        print(f"Warning: Only one class present in targets. ROC AUC set to {roc_auc}")


    return avg_loss, accuracy, roc_auc

    
num_epochs = 20 # Increase epochs, rely on saving the best
best_val_roc_auc = 0.0 # Initialize low for maximization
patience_counter = 0
patience = 5 # Number of epochs to wait for improvement before stopping

print(f"Starting training for {num_epochs} epochs...")

for epoch in range(1, num_epochs + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy, val_roc_auc = evaluate(model, val_loader, criterion, device)

    print(f'Epoch {epoch:02d}/{num_epochs}')
    print(f'  Train Loss: {train_loss:.4f}')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc (at 0.5): {val_accuracy:.4f}, Val ROC-AUC: {val_roc_auc:.4f}') # Label accuracy as potentially misleading

    # Save best model based on validation ROC-AUC
    if val_roc_auc > best_val_roc_auc:
        print(f'  Validation ROC-AUC improved ({best_val_roc_auc:.4f} --> {val_roc_auc:.4f}). Saving model...')
        best_val_roc_auc = val_roc_auc # Correctly update the best AUC score
        torch.save(model.state_dict(), 'best_transformer_model.pt')
        patience_counter = 0 # Reset patience counter
    else:
        patience_counter += 1
        print(f'  Validation ROC-AUC did not improve. Patience {patience_counter}/{patience}')

    # Early stopping
    if patience_counter >= patience:
        print(f'  Early stopping triggered after {epoch} epochs.')
        break # Exit the training loop

print("\nTraining finished.")

# Load best model (make sure the file exists)
print(f"Loading best model saved with ROC-AUC: {best_val_roc_auc:.4f}")
try:
    model.load_state_dict(torch.load('best_transformer_model.pt'))
except FileNotFoundError:
    print("Error: 'best_transformer_model.pt' not found. Was a model ever saved?")
    # Handle error appropriately - maybe exit or proceed with the last state of the model?

# Evaluate on test set
print('\nEvaluating on test set with the loaded best model...')
test_loss, test_accuracy, test_roc_auc = evaluate(model, test_loader, criterion, device)
print('\nTest set evaluation:')
# Remind that accuracy uses 0.5 threshold
print(f'Test Loss: {test_loss:.4f}, Test Acc (at 0.5): {test_accuracy:.4f}, Test ROC-AUC: {test_roc_auc:.4f}')

/Users/olivermarketos/miniconda3/envs/mybase/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Calculated pos_weight: 4.0000 (Negatives=3446, Positives=554)
Starting training for 20 epochs...
Epoch 01/20
  Train Loss: 0.8061
  Val Loss: 0.7923, Val Acc (at 0.5): 0.8538, Val ROC-AUC: 0.8231
  Validation ROC-AUC improved (0.0000 --> 0.8231). Saving model...
Epoch 02/20
  Train Loss: 0.7226
  Val Loss: 0.7163, Val Acc (at 0.5): 0.7997, Val ROC-AUC: 0.8282
  Validation ROC-AUC improved (0.8231 --> 0.8282). Saving model...
Epoch 03/20
  Train Loss: 0.6918
  Val Loss: 0.6991, Val Acc (at 0.5): 0.7712, Val ROC-AUC: 0.8378
  Validation ROC-AUC improved (0.8282 --> 0.8378). Saving model...
Epoch 04/20
  Train Loss: 0.6565
  Val Loss: 0.6754, Val Acc (at 0.5): 0.8010, Val ROC-AUC: 0.8478
  Validation ROC-AUC improved (0.8378 --> 0.8478). Saving model...
Epoch 05/20
  Train Loss: 0.6221
  Val Loss: 0.7523, Val Acc (at 0.5): 0.8415, Val ROC-AUC: 0.8375
  Validation ROC-AUC did not improve. Patience 1/5
Epoch 06/20
  Train Loss: 0.6174
  Val Loss: 0.7098, Val Acc (at 0.5): 0.8150, Val ROC-AU

In [170]:
# --- After loading the best model ---
print("\nFinding optimal threshold on validation set...")
model.eval() # Ensure model is in eval mode
val_logits = []
val_targets_list = []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).squeeze()
        val_logits.extend(outputs.cpu().numpy())
        val_targets_list.extend(y_batch.cpu().numpy())

val_logits = np.array(val_logits)
val_targets_np = np.array(val_targets_list).astype(int)
val_probs = 1 / (1 + np.exp(-val_logits))

# Method 1: Maximize F1 score
precision, recall, thresholds_pr = precision_recall_curve(val_targets_np, val_probs)
# Calculate F1 score, handling potential division by zero
f1_scores = 2 * recall * precision / (recall + precision + 1e-8)
# thresholds_pr correspond to precision/recall pairs, need to adjust index
optimal_idx_f1 = np.argmax(f1_scores)
optimal_threshold_f1 = thresholds_pr[optimal_idx_f1]
print(f"Optimal threshold based on max F1 score on validation set: {optimal_threshold_f1:.4f}")

# Method 2: Maximize Youden's J (sensitivity + specificity - 1)
fpr, tpr, thresholds_roc = roc_curve(val_targets_np, val_probs)
youden_j = tpr - fpr
optimal_idx_j = np.argmax(youden_j)
optimal_threshold_j = thresholds_roc[optimal_idx_j]
print(f"Optimal threshold based on max Youden's J on validation set: {optimal_threshold_j:.4f}")

# Choose one threshold (e.g., from F1)
optimal_threshold =  optimal_threshold_j # or optimal_threshold_f1 

# --- Now, evaluate on the TEST set using the optimal threshold ---

def evaluate_with_threshold(model, loader, criterion, device, threshold):
    model.eval()
    total_loss = 0
    all_outputs = [] # Store raw outputs (logits)
    all_targets = [] # Store targets

    with torch.no_grad():
        # (Same loop as before to get all_outputs and all_targets)
        for X_batch, y_batch in loader:
            # ... (identical data loading and model prediction) ...
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch).squeeze() # raw logits

            if outputs.dim() == 0: outputs = outputs.unsqueeze(0)
            if y_batch.dim() > outputs.dim(): y_batch = y_batch.squeeze()
            elif outputs.dim() > y_batch.dim(): outputs = outputs.squeeze()

            if outputs.shape != y_batch.shape:
                 print(f"Shape mismatch: outputs {outputs.shape}, y_batch {y_batch.shape}")
                 continue

            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            all_outputs.extend(outputs.cpu().numpy())
            all_targets.extend(y_batch.cpu().numpy())


    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)
    avg_loss = total_loss / len(loader)

    # Apply sigmoid and OPTIMAL threshold
    probs = 1 / (1 + np.exp(-all_outputs))
    preds_labels = (probs >= threshold).astype(int) # Use the optimal threshold

    all_targets = all_targets.astype(int)

    accuracy = accuracy_score(all_targets, preds_labels)
    roc_auc = roc_auc_score(all_targets, all_outputs) if len(np.unique(all_targets)) > 1 else 0.5

    # Calculate other metrics
    precision = precision_score(all_targets, preds_labels, zero_division=0)
    recall = recall_score(all_targets, preds_labels, zero_division=0)
    f1 = f1_score(all_targets, preds_labels, zero_division=0)

    return avg_loss, accuracy, roc_auc, precision, recall, f1


# Evaluate on test set using the found threshold
print(f'\nEvaluating on test set using threshold: {optimal_threshold:.4f}')
test_loss, test_accuracy, test_roc_auc, test_precision, test_recall, test_f1 = evaluate_with_threshold(
    model, test_loader, criterion, device, optimal_threshold
)

print('\nTest set evaluation (with optimized threshold):')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test ROC-AUC: {test_roc_auc:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Precision: {test_precision:.4f}')
print(f'Test Recall: {test_recall:.4f}')
print(f'Test F1-Score: {test_f1:.4f}')


Finding optimal threshold on validation set...
Optimal threshold based on max F1 score on validation set: 0.5538
Optimal threshold based on max Youden's J on validation set: 0.4659

Evaluating on test set using threshold: 0.4659

Test set evaluation (with optimized threshold):
Test Loss: 0.7475
Test ROC-AUC: 0.8116
Test Accuracy: 0.7675
Test Precision: 0.3480
Test Recall: 0.6752
Test F1-Score: 0.4593



Finding optimal threshold on validation set...
Optimal threshold based on max F1 score on validation set: 0.5538
Optimal threshold based on max Youden's J on validation set: 0.4659

Evaluating on test set using threshold: 0.5538

Test set evaluation (with optimized threshold):
Test Loss: 0.7475
Test ROC-AUC: 0.8116
Test Accuracy: 0.8225
Test Precision: 0.4181
Test Recall: 0.5453
Test F1-Score: 0.4733